# Iron Hackathon Data Cleaning

# Import libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

# Load datasets

In [2]:
# Load train.csv file
train_file = "../data/raw_data/train.csv"
df_train = pd.read_csv(train_file)

# Load test_no_class.csv file
test_file = "../data/raw_data/test_no_class.csv"
df_test_no_class = pd.read_csv(test_file)

# Explore Data

In [3]:
print(df_train.shape)
display(df_train.head())
print(df_test_no_class.shape)
display(df_test_no_class.head())

(102, 20)


,AGE,SEX,STEROID,ANTIVIRALS,FATIGUE,MALAISE,ANOREXIA,LIVER BIG,LIVER FIRM,SPLEEN PALPABLE,SPIDERS,ASCITES,VARICES,BILIRUBIN,ALK PHOSPHATE,SGOT,ALBUMIN,PROTIME,HISTOLOGY,Class
0,48,meale,1,2,1,1,2,2,1,2,1,1,1,4.80,123,157,2.7,31,2,DIE
1,51,Male,2,2,1,2,2,2,1,1,1,2,1,1.00,?,20,3.0,63,2,LIVE
2,40,m,1,2,1,2,2,2,1,2,2,2,2,0.60,62,166,4.0,63,1,LIVE
3,25,m,2,2,1,2,2,1,1,1,1,1,1,1.30,181,181,4.5,57,2,LIVE
4,34,M,1,2,1,2,2,1,1,2,1,2,2,1.00,72,46,4.4,57,1,LIVE


(53, 19)


,AGE,SEX,STEROID,ANTIVIRALS,FATIGUE,MALAISE,ANOREXIA,LIVER BIG,LIVER FIRM,SPLEEN PALPABLE,SPIDERS,ASCITES,VARICES,BILIRUBIN,ALK PHOSPHATE,SGOT,ALBUMIN,PROTIME,HISTOLOGY
0,39,Male,2,2,1,2,2,2,1,2,2,2,2,0.70,?,48,4.4,?,1
1,41,Feamle,2,1,1,1,1,2,2,2,2,2,2,0.70,81,53,5.0,74,1
2,28,maled,1,2,1,1,1,2,1,2,2,2,2,1.60,44,123,4.0,46,1
3,36,maled,1,2,1,1,1,2,1,2,2,2,2,1.00,?,45,4.0,57,1
4,32,M,2,2,2,2,2,2,2,2,2,2,2,0.70,102,64,4.0,90,1


In [4]:
# Show values por each column
#for col in df_train.columns:
#    display(df_train[col].value_counts(dropna=False))

# Replace & Clean

In [5]:
def replace_and_clean_df(df: pd.DataFrame) -> pd.DataFrame:

    df1 = df.copy()
    
    # Replace '?' with np.nan across the entire dataframe
    df1 = df1.replace('?', np.nan).apply(pd.to_numeric, errors='ignore')
    
    # Replace different representations of male and female with 'm' and 'f' respectively
    df1['SEX'] = df1['SEX'].apply(lambda x: x[0].lower())
    
    # Encode 'SEX' column
    df1['SEX'].replace({'m': 1, 'f': 2}, inplace=True)

    # Encode Class column
    if 'Class' in df1.columns:
        df1['Class'].replace({'DIE': 1, 'LIVE': 2}, inplace=True)

    return df1

In [6]:
clean_df = replace_and_clean_df(df_train)
clean_df_no_class = replace_and_clean_df(df_test_no_class)

## KNN Imputation

In [7]:
def impute_missing_values(df: pd.DataFrame) -> pd.DataFrame:

    df1 = df.copy()
    
    # Handle missing values using KNN
    imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
    
    knn_imputed_array = imputer.fit_transform(df1)
    
    knn_imputed_df = pd.DataFrame(knn_imputed_array, columns=df1.columns)

    # Round the values in "LIVER FIRM" and "LIVER BIG"
    knn_imputed_df["LIVER FIRM"] = np.round(knn_imputed_df["LIVER FIRM"])
    knn_imputed_df["LIVER BIG"] = np.round(knn_imputed_df["LIVER BIG"])

    # List of columns for mode imputation
    cols_mode = ["SPLEEN PALPABLE", "SPIDERS", "ASCITES", "VARICES", "STEROID"]
    
    mode_imputed_df = df1[cols_mode].copy()
    
    # Replace the missing values in cols_mode with the mode of the respective column
    for col in mode_imputed_df:
        mode_imputed_df[col].fillna(mode_imputed_df[col].mode()[0], inplace=True)
    
    # Drop cols_mode from imputed df
    knn_imputed_df.drop(columns=cols_mode, inplace=True)

    # Concat mode_imputed_df with knn_imputed_df
    clean_df = pd.concat([knn_imputed_df, mode_imputed_df], axis=1)
    
    # Move Class column to last position
    if 'Class' in df1.columns:
        clean_df['Class'] = clean_df.pop('Class')
    
    return clean_df

In [8]:
# Impute missing values
clean_imputed_df = impute_missing_values(clean_df)
clean_imputed_df_no_class = impute_missing_values(clean_df_no_class)

In [9]:
clean_imputed_df.head()

,AGE,SEX,ANTIVIRALS,FATIGUE,MALAISE,ANOREXIA,LIVER BIG,LIVER FIRM,BILIRUBIN,ALK PHOSPHATE,SGOT,ALBUMIN,PROTIME,HISTOLOGY,SPLEEN PALPABLE,SPIDERS,ASCITES,VARICES,STEROID,Class
0,48.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,4.8,123.0,157.0,2.7,31.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0
1,51.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,79.6,20.0,3.0,63.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0
2,40.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,0.6,62.0,166.0,4.0,63.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0
3,25.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,1.3,181.0,181.0,4.5,57.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0
4,34.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,72.0,46.0,4.4,57.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0


# Save clean dataframe as CSV

In [10]:
clean_imputed_df.to_csv('../data/clean_data/train_clean.csv', index=False)
clean_imputed_df_no_class.to_csv('../data/clean_data/test_no_class_clean.csv', index=False)